# The score seems to be low because it is too complicated, so I will model it by simply minimizing it this time.
# I think that Simple is Best 

In [ ]:
import seaborn as sns
import sys
import csv
import datetime
import operator
import joblib
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from collections import OrderedDict
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from scipy.stats import norm, skew, probplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from category_encoders.target_encoder import TargetEncoder
from sklearn.model_selection import StratifiedKFold

In [ ]:
df_train = pd.read_csv('../input/spaceship-titanic/train.csv')
df_test = pd.read_csv('../input/spaceship-titanic/test.csv')

In [ ]:
df_train["Age"].fillna(df_train["Age"].median(), inplace=True)
df_train["FoodCourt"].fillna(df_train["FoodCourt"].median(), inplace=True)
df_train["ShoppingMall"].fillna(df_train["ShoppingMall"].median(), inplace=True)
df_train["Spa"].fillna(df_train["Spa"].median(), inplace=True)
df_train["VRDeck"].fillna(df_train["VRDeck"].median(), inplace=True)
df_train["RoomService"].fillna(df_train["RoomService"].median(), inplace=True)
# Categorical Values
df_train["HomePlanet"].fillna(df_train["HomePlanet"].mode().values[0], inplace=True)
df_train["CryoSleep"].fillna(df_train["CryoSleep"].mode().values[0], inplace=True)
df_train["Destination"].fillna(df_train["Destination"].mode().values[0], inplace=True)
df_train["VIP"].fillna(df_train["VIP"].mode().values[0], inplace=True)

In [ ]:
df_test["Age"].fillna(df_test["Age"].median(), inplace=True)
df_test["FoodCourt"].fillna(df_test["FoodCourt"].median(), inplace=True)
df_test["ShoppingMall"].fillna(df_test["ShoppingMall"].median(), inplace=True)
df_test["Spa"].fillna(df_test["Spa"].median(), inplace=True)
df_test["VRDeck"].fillna(df_test["VRDeck"].median(), inplace=True)
df_test["RoomService"].fillna(df_test["RoomService"].median(), inplace=True)
# Categorical Values
df_test["HomePlanet"].fillna(df_test["HomePlanet"].mode().values[0], inplace=True)
df_test["CryoSleep"].fillna(df_test["CryoSleep"].mode().values[0], inplace=True)
df_test["Destination"].fillna(df_test["Destination"].mode().values[0], inplace=True)
df_test["VIP"].fillna(df_test["VIP"].mode().values[0], inplace=True)

In [ ]:
df_train['Cabin'].fillna('Z/9999/Z', inplace=True)
df_test['Cabin'].fillna('Z/9999/Z', inplace=True)

# New features - training set
df_train['Cabin_deck'] = df_train['Cabin'].apply(lambda x: x.split('/')[0])
df_train['Cabin_number'] = df_train['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)
df_train['Cabin_side'] = df_train['Cabin'].apply(lambda x: x.split('/')[2])

# New features - test set
df_test['Cabin_deck'] = df_test['Cabin'].apply(lambda x: x.split('/')[0])
df_test['Cabin_number'] = df_test['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)
df_test['Cabin_side'] = df_test['Cabin'].apply(lambda x: x.split('/')[2])

# Put Nan's back in (we will fill these later)
df_train.loc[df_train['Cabin_deck']=='Z', 'Cabin_deck']=np.nan
df_train.loc[df_train['Cabin_number']==9999, 'Cabin_number']=np.nan
df_train.loc[df_train['Cabin_side']=='Z', 'Cabin_side']=np.nan
df_test.loc[df_test['Cabin_deck']=='Z', 'Cabin_deck']=np.nan
df_test.loc[df_test['Cabin_number']==9999, 'Cabin_number']=np.nan
df_test.loc[df_test['Cabin_side']=='Z', 'Cabin_side']=np.nan

# Drop Cabin (we don't need it anymore)
df_train.drop('Cabin', axis=1, inplace=True)
df_test.drop('Cabin', axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

random_state_val =42
test_size_val =0.2
train,validation = train_test_split(df_train, test_size = test_size_val, random_state = random_state_val)

In [ ]:
y_nm = 'Transported'

df_train_x = train.drop(y_nm, axis = 1)
df_train_y = pd.DataFrame(train[y_nm])

df_val_x = validation.drop(y_nm, axis = 1)
df_val_y = pd.DataFrame(validation[y_nm])

df_test_x = df_test

In [ ]:
num_cols = [col for col in df_train_x.columns if df_train_x[col].dtype in ["float16","float32","float64"]]
cat_cols = [col for col in df_train_x.columns if df_train_x[col].dtype not in ["float16","float32","float64"]]

In [ ]:
y = train["Transported"].copy()

for cols in cat_cols:
    enc = TargetEncoder(cols=[cols])
    df_train_x = enc.fit_transform(df_train_x, y)
    df_val_x = enc.transform(df_val_x)
    df_test_x = enc.transform(df_test_x)

In [ ]:
scaler = QuantileTransformer()
scaler.fit(df_train_x)
df_train_x = pd.DataFrame(scaler.transform(df_train_x))
df_val_x = pd.DataFrame(scaler.transform(df_val_x))
df_test_x = pd.DataFrame(scaler.transform(df_test_x))

In [ ]:
XGBClassifier = xgb.XGBClassifier(max_depth = 8,
                                 learning_rate = 0.001,
                                 n_estimators = 10000,
                                 objective = 'binary:logistic',
#                                  tree_method = 'gpu_hist',
#                                  booster = 'gbtree',
                                 gamma = 0.64,
                                 max_delta_step = 3,
                                 min_child_weight = 7,
                                 subsample = 0.7,
                                 colsample_bytree = 0.8,
                                 n_jobs = -1
                                 )

In [ ]:
start = datetime.datetime.now()
xgb = XGBClassifier.fit(df_train_x.values,
                       df_train_y.values.ravel(),
                       eval_set = [(df_train_x.values, df_train_y), (df_val_x.values, df_val_y)], 
                       eval_metric = 'auc',
                       early_stopping_rounds = 200,
                       verbose = True)
end = datetime.datetime.now()
end-start

In [ ]:
fi_vals = xgb.get_booster().get_score(importance_type = 'weight')
fi_dict = {df_train_x.columns[i]:float(fi_vals.get('f'+str(i),0.)) for i in range(len(df_train_x.columns))}
feature_importance_ = sorted(fi_dict.items(), key=operator.itemgetter(1), reverse=True)
feature_importance_result = OrderedDict(feature_importance_)

importance = pd.DataFrame(feature_importance_)
importance.columns = ['feature','weight']
importance.head(10)

In [ ]:
importance_ten = importance[:10]
importance_ten.set_index('feature').sort_values(by='weight').plot(kind='barh', figsize=(5, 5))

In [ ]:
fpr, tpr, _ = roc_curve(df_val_y, xgb.predict_proba(df_val_x.values)[:, 1])
roc_auc = auc(fpr, tpr)

result_lst =[]
max_roc_auc =0.
opt_threshold =0.
val_y_prob = xgb.predict_proba(df_val_x.values)[:, 1]

for n in range(0,50):
    threshold = round(((n+1)*0.01),2)
    pred_yn = val_y_prob.copy()
    pred_yn = np.where(pred_yn > threshold, 1., 0.)
    
    result_dict = {}
    precision, recall, f1_score, support = precision_recall_fscore_support(df_val_y.values.ravel(), pred_yn, average='binary')
    accuracy = accuracy_score(df_val_y.values.ravel(), pred_yn)
    kappa = cohen_kappa_score(df_val_y.values.ravel(), pred_yn)
    
    result_dict ={'Threshold': threshold, 'Accuracy': round(accuracy,4), 'Precision': round(precision,4), 'Recall': round(recall,4), 'F1_Score': round(f1_score,4),'roc_auc': round(roc_auc,4), 'Kappa': round(kappa,4)}
    result_lst.append(result_dict)
    
    if max_roc_auc <= roc_auc:
        max_roc_auc = roc_auc
        opt_threshold = threshold
        
    confMat = confusion_matrix(df_val_y.values.ravel(), pred_yn, labels=[1,0])
    
matric_df = pd.DataFrame(result_lst, columns=['Threshold','Accuracy', 'Precision', 'Recall', 'F1_Score','roc_auc' ,'Kappa'])
matric_df.to_csv('REC_scores.csv',sep=',', header=True, index=False, encoding='UTF-8')

print('Max roc_auc =%f, optimized_threshold=%f'%(max_roc_auc, opt_threshold))
print('Complete')

In [ ]:
predict_xgb = xgb.predict_proba(df_train_x.values)[:,1]
pred_train = np.where(predict_xgb > opt_threshold, 1., 0.)

tp, fn, fp, tn = confusion_matrix(df_train_y.values.ravel(), pred_train, labels=[1,0]).ravel()

conf_matrix = pd.DataFrame(
    confusion_matrix(df_train_y.values.ravel(), pred_train),
    columns=['Predicted Value 0', 'Predicted Value 1'],
    index=['True Value 0', 'True Value 1']
)

print("1. Counfusion Matrix")
print(conf_matrix.T)
print("")

print("2. Classification Report")
print(classification_report(df_train_y.values.ravel(), pred_train))

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, _ = roc_curve(df_train_y.values.ravel(), predict_xgb)

import matplotlib.pyplot as plt
roc_auc = auc(fpr, tpr)

# Plot of a ROC curve for a specific class
plt.figure()
plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.3f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
df_test.head()

In [ ]:
pred_test = xgb.predict_proba(df_test_x.values)[:,1]

test_result= pd.DataFrame(pred_test)
test_result.columns = ['Transported']
predict = test_result['Transported']
Id_No = df_test['PassengerId']
submission = pd.DataFrame({'PassengerId': Id_No, 'Transported': predict})
# submission['Transported'] = submission['Transported'].astype('int')
submission['Transported'] = (submission['Transported']>=0.5).astype(int)
submission=submission.replace({0:False, 1:True})
submission.to_csv('submission.csv', index=False)
submission.head()